In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [2]:
import numpy as np

In [3]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

print(USE_CUDA,device)

True cuda


# Load Dictionary 

In [4]:
import pickle

In [5]:
dict_dir = "../dict/" 

#単語ファイルロード
with open(dict_dir + 'words.pickle', 'rb') as ff :
    words=pickle.load(ff)         

#Encoder Inputデータをロード
with open(dict_dir + 'e.pickle', 'rb') as f :
    e = pickle.load(f)

#Decoder Inputデータをロード
with open(dict_dir + 'd.pickle', 'rb') as g :
    d = pickle.load(g)

#ラベルデータをロード
with open(dict_dir + 't.pickle', 'rb') as h :
    t = pickle.load(h)

#maxlenロード
with open(dict_dir + 'maxlen.pickle', 'rb') as maxlen :
    [maxlen_e, maxlen_d] = pickle.load(maxlen)

In [6]:
n_split=int(e.shape[0]*0.95)            #訓練データとテストデータを95:5に分割
e_train,e_test=np.vsplit(e,[n_split])   #エンコーダインプットデータを訓練用とテスト用に分割
d_train,d_test=np.vsplit(d,[n_split])   #デコーダインプットデータを訓練用とテスト用に分割
t_train,t_test=np.vsplit(t,[n_split])   #ラベルデータを訓練用とテスト用に分割

In [7]:
e.shape

(4830, 50, 1)

In [8]:
d.shape

(4830, 50, 1)

In [9]:

print( e[0,:,0].astype(int) )
print( e[1,:,0].astype(int) )

print( d[0,:,0].astype(int) )
print( d[1,:,0].astype(int) )


[   1 5276 5439  531    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[   1  331 1745  217 2690 1420 1486 4903  325 2153 4798 3806  706  690
   67 5626 2131 4553    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[   2  509 5444    8  933 3567 4263 5937 6010 5937 2605 2499 3734 5210
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[   2  331 1743  217 5692 1420    3 3622 4249  294 6103 4963 3806  592
 3705  971    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  

In [10]:
words[:10]

['\t', 'REQREQ', 'RESRES', 'UNK', 'é', 'ŧ', 'μ', 'σ', 'ω', 'г']

In [11]:
words[0] = "PAD"
words[1] = "SOS"
words[2] = "EOS"

In [12]:
PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"

SyntaxError: invalid syntax (<ipython-input-12-ebbc5329ca12>, line 1)

In [13]:
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        #self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    #def addSentence(self, sentence):
    #    for word in sentence.split(' '):
    #        self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            #self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        #else:
            #self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [14]:
voc = Voc("tweet")


In [15]:
# Add words from words data
for w in words[3:]:
    voc.addWord(w)


In [16]:
MAX_LENGTH = 10

print("encoder shape : ",e.shape)
new_enc = np.squeeze(e)
new_enc= new_enc[:,1:(MAX_LENGTH+1)].astype(int)
print("encoder new shape : ", new_enc.shape)

print("decoder shape : ",d.shape)
new_dec = np.squeeze(d)
new_dec= new_dec[:,1:(MAX_LENGTH+1)].astype(int)
print("decoder new shape : ",new_dec.shape)



encoder shape :  (4830, 50, 1)
encoder new shape :  (4830, 10)
decoder shape :  (4830, 50, 1)
decoder new shape :  (4830, 10)


In [17]:
test_10x10 = new_enc[:10,:].copy()

## replace last character of sentences with 2(EOS_token) for encoder/decoder

In [18]:
def assign_EOS_token(row):

    last_index=( MAX_LENGTH - np.sum(row == 0) )
    row[(last_index-1)] = 2
    return row

In [19]:
nenc = np.apply_along_axis(assign_EOS_token, 1, new_enc)
ndec = np.apply_along_axis(assign_EOS_token, 1, new_dec)


# Build Batch Process 

In [20]:
nenc[:10,:]

array([[5276, 5439,    2,    0,    0,    0,    0,    0,    0,    0],
       [ 331, 1745,  217, 2690, 1420, 1486, 4903,  325, 2153,    2],
       [1717, 5776, 4386, 4345, 2450,  758, 5105, 2027, 3617,    2],
       [ 494, 3268, 4771, 5580,  390,    2,    0,    0,    0,    0],
       [1612, 1683, 4642,  986, 3190, 2386, 4390, 4619, 1682,    2],
       [3647, 1470, 2576, 3089,    3, 3647, 2771, 5436, 5504,    2],
       [ 784, 2085, 2770, 5504, 1492, 5586,    3,    2,    0,    0],
       [2148,  705,  742, 3692, 5659, 5442,    2,    0,    0,    0],
       [1705, 3954, 2687,  552, 1004,    3,  544, 4963,  552,    2],
       [   3,  670, 5444, 1218, 2302,    2,    0,    0,    0,    0]])

In [21]:
ndec[:10,:]

array([[ 509, 5444,    8,  933, 3567, 4263, 5937, 6010, 5937,    2],
       [ 331, 1743,  217, 5692, 1420,    3, 3622, 4249,  294,    2],
       [1689, 2163,  548, 5566, 3147, 1004, 1207,    3, 5021,    2],
       [1858,  502,    3, 2120, 3954, 3268,    2,    0,    0,    0],
       [1689, 2163,  548, 5566, 3147, 1004, 1207,    3, 5021,    2],
       [3647, 1470, 2576, 3089,    3, 3647, 2771, 5436, 5504,    2],
       [ 784, 2871, 5504, 5657, 2326,    3,  933, 3651, 4856,    2],
       [ 331,    3, 2104,   48, 5732, 5659, 4841, 1743,    2,    0],
       [3954,    2,    0,    0,    0,    0,    0,    0,    0,    0],
       [   3, 6071, 2469, 3978, 5898, 5593, 2626,    3,  288,    2]])

In [38]:
print( batch_index )
print( nenc[batch_index])

[1496 3329 2541 1406 1684]
[[4914 2719 2149 1167 5357 4692    2    0    0    0]
 [ 933 2802 2030  531  420  420 3254  263 3793    2]
 [2748 1758 5514 5376  342 3967  531 3278 1705    2]
 [ 928 5834 6098    2    0    0    0    0    0    0]
 [4007 3791   65  632 3845 5147 3821 1636 5788    2]]


In [49]:
nenc_batch = nenc[batch_index].copy()
sort_index = np.argsort( (nenc_batch > 0).sum(axis=1) )[::-1]
print(nenc_batch[sort_index,:])
print()
ndec_batch = ndec[batch_index].copy()
print(ndec_batch[sort_index,:])

[[4007 3791   65  632 3845 5147 3821 1636 5788    2]
 [2748 1758 5514 5376  342 3967  531 3278 1705    2]
 [ 933 2802 2030  531  420  420 3254  263 3793    2]
 [4914 2719 2149 1167 5357 4692    2    0    0    0]
 [ 928 5834 6098    2    0    0    0    0    0    0]]

[[ 176 2728 3967 3825    2    0    0    0    0    0]
 [ 231  342 3967 1794 3380    3  404 3967 4546    2]
 [ 670 3380 2645  526  994 4205  494  562 2147    2]
 [4914 2719 2149 1167 5357 4692    2    0    0    0]
 [1165    2    0    0    0    0    0    0    0    0]]


In [53]:
length = (nenc_batch > 0).sum(axis=1)[sort_index]

print(length)


[10 10 10  7  4]
[ 4 10 10 10  7]


In [55]:
max_target_len = np.max( (ndec_batch > 0).sum(axis=1) )
print(max_target_len)

10


In [48]:
torch.BoolTensor( (ndec_batch > 0)[sort_index,:].transpose() ) 

tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True, False],
        [False,  True,  True,  True, False],
        [False,  True,  True,  True, False],
        [False,  True,  True, False, False],
        [False,  True,  True, False, False],
        [False,  True,  True, False, False]])

In [50]:
torch.LongTensor( ndec_batch[sort_index,:].transpose() )

tensor([[ 176,  231,  670, 4914, 1165],
        [2728,  342, 3380, 2719,    2],
        [3967, 3967, 2645, 2149,    0],
        [3825, 1794,  526, 1167,    0],
        [   2, 3380,  994, 5357,    0],
        [   0,    3, 4205, 4692,    0],
        [   0,  404,  494,    2,    0],
        [   0, 3967,  562,    0,    0],
        [   0, 4546, 2147,    0,    0],
        [   0,    2,    2,    0,    0]])

In [68]:
test_input_tensor = torch.LongTensor( nenc_batch[sort_index,:].transpose() )

print( [voc.index2word[i] for i in test_input_tensor[:,0].numpy()] )

['梅雨明け', '早い', 'いい', 'まぁ', '暑い', '苦手', '昨日', '久しぶり', '都会', 'EOS']


In [122]:
def batch2TrainData(in_encoder, in_decoder, batch_size=5):
    
    r = np.random.permutation( in_encoder.shape[0] )

    #training_batches = []

    for i in range(1):
        batch_index = r[:batch_size]

        nenc_batch = in_encoder[batch_index].copy()
        sort_index = np.argsort( (nenc_batch > 0).sum(axis=1) )[::-1]
        inp = torch.LongTensor( nenc_batch[sort_index,:].transpose() )
        
        ndec_batch = in_decoder[batch_index].copy()
        mask = torch.BoolTensor( (ndec_batch > 0)[sort_index,:].transpose() ) 
        
        output = torch.LongTensor( ndec_batch[sort_index,:].transpose() )
        
        lengths = torch.tensor( (nenc_batch > 0).sum(axis=1)[sort_index] )

        max_target_len = np.max( (ndec_batch > 0).sum(axis=1) )
        
        #training_batches.append([inp, lengths, output, mask, max_target_len])
        

    return inp, lengths, output, mask, max_target_len

In [123]:
n_iteration = 4000
batch_size = 64
mybatches = [batch2TrainData(nenc,ndec,batch_size)
                      for _ in range(n_iteration)]


print(len(mybatches))

4000


In [124]:
input_variable, lengths, target_variable, mask, max_target_len = mybatches[0]

In [125]:
lengths

tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10,  9,  9,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,  6,  6,  5,  5,  4,
         4,  4,  4,  3,  3,  3,  2,  2,  1,  1])

# Encoder/Decoder Seq2Seq model

In [66]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden


# Attention Layer 

In [69]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [70]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# Mask Loss

In [71]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()


In [72]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [97]:
print(nenc.shape, ndec.shape)

(4830, 10) (4830, 10)


# Training 

In [127]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    #training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
    #                  for _ in range(n_iteration)]

    training_batches = [batch2TrainData(nenc,ndec,batch_size)
                      for _ in range(n_iteration)]
    print("training batch total size ---> ",  len(training_batches) )
    #print("example of batches [0]", training_batches[0])
    
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [89]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [90]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

# Start to train model 

In [91]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [128]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 500
save_every = 500

save_dir = "save"
corpus_name = "tweet"

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
pairs = []
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
training batch total size --->  4000
Initializing ...
Training...
Iteration: 500; Percent complete: 12.5%; Average loss: 0.1107
Iteration: 1000; Percent complete: 25.0%; Average loss: 0.0879
Iteration: 1500; Percent complete: 37.5%; Average loss: 0.0884
Iteration: 2000; Percent complete: 50.0%; Average loss: 0.0739
Iteration: 2500; Percent complete: 62.5%; Average loss: 0.0747
Iteration: 3000; Percent complete: 75.0%; Average loss: 0.0730
Iteration: 3500; Percent complete: 87.5%; Average loss: 0.0726
Iteration: 4000; Percent complete: 100.0%; Average loss: 0.0721
